In [4]:
import pygame
import time
pygame.font.init()

In [5]:
# grid class
class Grid :
    board = [
       [7, 8, 0, 4, 0, 0, 1, 2, 0],
       [6, 0, 0, 0, 7, 5, 0, 0, 9],
       [0, 0, 0, 6, 0, 1, 0, 7, 8],
       [0, 0, 7, 0, 4, 0, 2, 6, 0],
       [0, 0, 1, 0, 5, 0, 9, 3, 0],
       [9, 0, 4, 0, 6, 0, 0, 0, 5],
       [0, 7, 0, 3, 0, 0, 0, 1, 2],
       [1, 2, 0, 0, 0, 7, 4, 0, 0],
       [0, 4, 9, 2, 0, 6, 0, 0, 7]
]            

    # define constructor
    def __init__(self, rows, cols, width, height, win) :
        self.rows = rows
        self.cols = cols
        self.width = width
        self.height = height
        
        # deifines cubes with Cube class
        # pass boards[i][j] as value
        
        self.cubes = [[Cube(self.board[i][j], i, j, width, height) for j in range(cols)] for i in range(rows)]
        self.win = win
        
        # create model
        
        self.model = None
        
        # update model
        
        self.update_model()
        self.selected = None
        

        
        
    # Define update_model
    
    def update_model(self):
        self.model = [[self.cubes[i][j].value for j in range(self.cols)] for i in range(self.rows)]
            
    # Define select method( Select the given row,col & unselect all other)
    
    def select(self, row, col) :
        # Reset all
        for i in range(self.rows) :
            for j in range(self.cols) :
                self.cubes[i][j].selected = False
                    
        # Select given row, col
        
        self.cubes[row][col].selected = True
        self.selected = (row,col)
            
    # Define place to place the  val on selected row,col
    
    def place(self, val) :
        row, col = self.selected
        if self.cubes[row][col] !=  0 :
            self.cubes[row][col].set(val)
            self.update_model()
            
            # Check if model valid & Solved
            
            if valid(self.model, val, (row, col)) and self.solve() :
                return True
            else :
                self.cubes[row][col].set(0)
                self.cubes[row][col].set_temp(0)
                self.update_model()
                return False
            
    # Define Solve GUI
    # Find empty cube
    
    def solve_gui(self) :                                                     
        find = find_empty(self.model)
        # Base Case (if NO cell is empty means model is solved)
        if not find:
            return True
        else :
            row, col = find
            
        for i in range(1, 10) :
            # Check for validity
            if valid(self.model, i, (row,col)) :
                # Assign value of i 
                self.model[row][col] = i
                # Set value in the cube
                self.cubes[row][col].set(i)
                # Draw the changes using draw_chage() method , True -> as number is valid
                self.cubes[row][col].draw_change(self.win, True)
                # Update the model
                self.update_model()
                # Update the display
                pygame.display.update()
                # Put a time delay
                pygame.time.delay(100)
                
                # check if Solution is Right with RECURSSION
                if self.solve_gui() :
                    return True
                
                # If solution is not currect Set and Draw 0
                self.model[row][col] = 0
                self.cubes[row][col].set(0)
                # False -> as solution is False
                self.cubes[row][col].draw_change(self.win, False)
                pygame.display.update()
                pygame.time.delay(100)
                
        return False
        
    # Define solve
    
    def solve(self) :
        # find empty cube
        find = find_empty(self.model)
        # Base case
        if not find :
            return True
        else :
            row, col = find
            
        for i in range(1,10) :
            if valid(self.model, i, (row,col)) :
                self.model[row][col] = i
                    
                if self.solve() :
                    return True
                    
                self.model[row][col] = 0
                
        return False
        
    # Check is the board  is solved
    
    def is_finished(self) :
        for i in range(self.rows) :
            for j in range(self.cols) :
                if self.cubes[i][j].value == 0 :
                    return False
                    
        return True
                
    # Clear the selected 
    
    def clear(self) :
        row, col = self.selected
            
        if self.cubes[row][col].value == 0 :
            self.cubes[row][col].set_temp(0)
            
            
    def click(self, pos) :
        if pos[0] < self.width and pos[1] < self.height :
            gap = self.width / 9
            x = pos[0] // gap
            y = pos[1] // gap
            return(int(y), int(x))
        else :
            return None
        
    def sketch(self, val) :
        row, col = self.selected
        self.cubes[row][col].set_temp(val)
    
    # Define draw 
    def draw(self) :
        gap = self.width / 9
        for i in range(self.rows+1):
            if i %3 == 0 and i != 0 :
                thick = 4
            else :
                thick = 1
                
            #draw a straight line
            #line(surface, color, start_pos, end_pos, width) -> Rect
            
            pygame.draw.line(self.win, (0, 0, 0), (0, i*gap), (self.width, i*gap), thick)
            pygame.draw.line(self.win, (0, 0, 0), (i*gap, 0),(i*gap, self.height), thick)
            
        # Draw Cubes
        
        for i in range(self.rows):
            for j in range(self.cols):
                self.cubes[i][j].draw(self.win)

class Cube :
    rows = 9
    cols = 9
    
    def __init__(self, value, row, col, width, height):
        self.value = value
        self.row = row
        self.col = col
        self.width = width
        self.height = height
        # define temp
        self.temp = 0
        #define selected
        self.selected = False

    # Define set       
    def set(self, val) :
        self.value = val
        
        
    # Define draw_change method
    
    def draw_change(self, win, g = True) :
        fnt = pygame.font.SysFont("comicsans", 40)
        
        # define each gap
        gap = self.width/9
        
        # self.row, self.col -> cube return by valid method
        y = self.row * gap
        x = self.col * gap
        
        # Draw ractangle pygame.draw.ract(); rect(surface, color, rect, width=0) ;
        #width = (optional) used for line thikness ; ract =position and dimention
        
        pygame.draw.rect(win, (255, 255, 255), (x, y, gap, gap), 0)
        
        # Render the text ; Create a text surface object
        text = fnt.render(str(self.value), 1, (0, 0, 0))
        
        # blit the text into the display window
        # .get_width -> inblid function retun width of the text
        win.blit(text, (x + (gap/2 - text.get_width() / 2 ), y + (gap/2 - text.get_height() /2)))
        if g:
            pygame.draw.rect(win, (0, 255, 0), (x, y, gap, gap), 3)
        else:
            pygame.draw.rect(win, (255, 0, 0), (x, y, gap, gap), 3)

    
    # Define set temp
    
    def set_temp(self, val) :
        self.temp = val
        
    def draw(self, win):
        fnt = pygame.font.SysFont("comicsans", 40)

        gap = self.width / 9
        x = self.col * gap
        y = self.row * gap

        if self.temp != 0 and self.value == 0:
            text = fnt.render(str(self.temp), 1, (128,128,128))
            win.blit(text, (x+5, y+5))
        elif not(self.value == 0):
            text = fnt.render(str(self.value), 1, (0, 0, 0))
            win.blit(text, (x + (gap/2 - text.get_width()/2), y + (gap/2 - text.get_height()/2)))

        if self.selected:
            pygame.draw.rect(win, (255,0,0), (x,y, gap ,gap), 3)
            
            
def find_empty(bo) :
    for i in range(len(bo)) :
        for j in range(len(bo[0])) :
            if bo[i][j] == 0 :
                return(i, j)
        
    return None


def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True


def redraw_window(win, board, time, strikes) :
    # draw  a white window
    win.fill((255, 255, 255))
    # Draw Time
    fnt = pygame.font.SysFont("cosmicsans", 40)
    text = fnt.render("Time" + format_time(time), 1, (0, 0, 0))
    win.blit(text, (540 - 130, 555))
    # Draw Strikes
    text = fnt.render("X" * strikes, 1, (255, 0, 0))
    win.blit(text, (20, 560))
    # Draw grid and Line
    board.draw()
    
def format_time(secs) :
    sec = secs %60
    minute = sec //60
    hour = minute //60
    mat = " " + str(minute) + ":" + str(sec)
    return(mat)
        
        
        
def main() :
    win = pygame.display.set_mode((540, 600))
    pygame.display.set_caption("Sudoku Solver")
    
    # Create a grid
    board = Grid(9, 9, 540, 540, win)
    key = None
    run = True
    # None start time
    start = time.time()
    
    # define strikes 
    strikes = 0
    
    # define a infine loop
    while run :
        
        # calculate playtime
        play_time = round(time.time() - start)
        
        # Get all the events inside your game window
        for event in pygame.event.get() :
            # Define QUIT
            if event.type == pygame.QUIT :
                run = False
    
            # Capture all KEYDOWN type events
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1 :
                    key = 1
                if event.key == pygame.K_2 :
                    key = 2
                if event.key == pygame.K_3 :
                    key = 3
                if event.key == pygame.K_4 :
                    key = 4
                if event.key == pygame.K_5 :
                    key = 5
                if event.key == pygame.K_6 :
                    key = 6
                if event.key == pygame.K_7 :
                    key = 7
                if event.key == pygame.K_8 :
                    key = 8
                if event.key == pygame.K_9 :
                    key = 9
                
                # capture del KEYDOWN
                if event.key == pygame.K_DELETE :
                    # Clear the board
                    board.clear()
                    #set key to None
                    key = None
                 
                # Capture space KEYDOWN
                if event.key == pygame.K_SPACE :
                    #solve the board
                    board.solve_gui()
                    
                # Capture return key
                if event.key == pygame.K_RETURN :
                    i, j = board.selected
                    if board.cubes[i][j].temp != 0 :
                        if board.place(board.cubes[i][j].temp) :
                            print('Success')
                        else :
                            print('Wrong')
                            strikes += 1
                        key = None
                        
                        if board.is_finished() :
                            print('Game Over')
                            
            if event.type == pygame.MOUSEBUTTONDOWN :
                pos = pygame.mouse.get_pos()
                # Get clicked row,col
                clicked = board.click(pos)
                if clicked :
                    board.select(clicked[0], clicked[1])
                    key = None
            
        if board.selected and key != None :
            board.sketch(key)
            
        redraw_window(win, board, play_time, strikes)
        pygame.display.update()

main()
pygame.quit()
            

                             
        